<a href="https://www.kaggle.com/code/mortezaheidari/ctm-contextualized-topic-modeling?scriptVersionId=154773669" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Introduction: Unveiling the Significance of Topic Modeling

Topic modeling, a powerful analytical tool, enables the extraction of meaningful patterns embedded within textual data. Its primary function is to facilitate the comprehension of large document collections by uncovering latent distributions of topics. This process simplifies the task of gaining insights into the content without the need to read each document individually. Through the lens of topic models, we can obtain a bird's eye view of the textual landscape, making it an invaluable asset for anyone dealing with extensive sets of documents.

## Exploring the Functionality of Topic Models

In practical terms, topic models provide a structured approach to understanding the recurring themes present in a document collection. Rather than delving into individual documents one by one, topic modeling allows for the extraction of prevalent topics, offering a condensed summary of the overall content. By identifying patterns and themes that repeat across documents, this analytical technique transforms the daunting task of reviewing large datasets into a more manageable and insightful process. It becomes an indispensable aid for researchers, analysts, and anyone seeking a comprehensive overview of complex textual information.

## Applications and Benefits of Topic Modeling

Beyond the efficient summarization of document collections, topic modeling finds applications in various fields. It is widely used in natural language processing, information retrieval, and content recommendation systems. By revealing the underlying structures of textual data, topic models contribute to enhanced decision-making processes and foster a deeper understanding of the subject matter. Whether applied in academic research, business intelligence, or data analysis, topic modeling stands as a versatile tool that unlocks valuable insights from the vast realm of textual information.


#### Ref1: Bianchi, F., Terragni, S., Hovy, D., Nozza, D., & Fersini, E. (2021). Cross-lingual Contextualized Topic Models with Zero-shot Learning. European Chapter of the Association for Computational Linguistics (EACL). https://arxiv.org/pdf/2004.07737/
#### Ref2: https://github.com/MilaNLProc/contextualized-topic-models/tree/master

## Getting the data from Wikipedia for training and validation

In [1]:
%%capture
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_unprep.txt
!wget https://raw.githubusercontent.com/vinid/data/master/dbpedia_sample_abstract_20k_prep.txt

### Installing Contextualized Topic Models


In [2]:
%%capture
!pip install contextualized-topic-models
!pip install torch torchvision

In [3]:
conda install -c conda-forge library_name

Retrieving notices: ...working... done
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - library_name

Current channels:

  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch
  - https://conda.anaconda.org/rapidsai/linux-64
  - https://conda.anaconda.org/rapidsai/noarch
  - https://conda.anaconda.org/nvidia/linux-64
  - https://conda.anaconda.org/nvidia/noarch
  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the t

### Loading libraries:

In [4]:
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import bert_embeddings_from_file, TopicModelDataPreparation
from contextualized_topic_models.datasets.dataset import CTMDataset
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO
from gensim.corpora.dictionary import Dictionary
from gensim.models import ldamodel 
import os
import numpy as np
import pickle
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Let's read our data files and store the documents as lists of strings

In [5]:
with open("dbpedia_sample_abstract_20k_prep.txt", 'r') as fr_prep:
  text_training_preprocessed = [line.strip() for line in fr_prep.readlines()]

with open("dbpedia_sample_abstract_20k_unprep.txt", 'r') as fr_unprep:
  text_training_not_preprocessed = [line.strip() for line in fr_unprep.readlines()]

### NOTE: Make sure that the lenghts of the two lists of documents are the same and the index of a not preprocessed document corresponds to the index of the same preprocessed document.

In [6]:
assert len(text_training_preprocessed) == len(text_training_not_preprocessed)

print(text_training_not_preprocessed[0])
print(text_training_preprocessed[0])

The Mid-Peninsula Highway is a proposed freeway across the Niagara Peninsula in the Canadian province of Ontario. Although plans for a highway connecting Hamilton to Fort Erie south of the Niagara Escarpment have surfaced for decades,it was not until The Niagara Frontier International Gateway Study was published by the Ministry
mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry


# sSplit the documents into training and testing

In [7]:
training_bow_documents = text_training_preprocessed[0:15000]
training_contextual_document = text_training_not_preprocessed[0:15000]

testing_bow_documents = text_training_preprocessed[15000:]
testing_contextual_documents = text_training_not_preprocessed[15000:]

## Creating the Training Dataset
#### Let's pass our files with preprocess data to our TopicModelDataPreparation object. This object takes care of creating the bag of words for you and of obtaining the contextualized BERT representations of documents. This operation allows us to create our training dataset.

In [8]:
tp = TopicModelDataPreparation("bert-base-nli-mean-tokens")
# qt = TopicModelDataPreparation("all-mpnet-base-v2") # TODO: you can change this data preparation 
training_dataset = tp.fit(training_contextual_document, training_bow_documents)

.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/75 [00:00<?, ?it/s]

Why do we use the preprocessed text here? We need text without punctuation to build the bag of word. Also, we might want only to have the most frequent words inside the BoW. Too many words might not help.

And what about the unpreprocessed text? We provide unpreprocessed text as the input for BERT (or the contextualized model of your choice) to let the model output more accurate document representations.

In [9]:
tp.vocab[:10]

array(['abbreviated', 'academic', 'academy', 'access', 'according',
       'achieved', 'acquired', 'acre', 'acres', 'across'], dtype=object)

### Combined Topic Model
Here is how you can use the CombinedTM. This is a standard topic model that also uses contextualized embeddings. The good thing about CombinedTM is that it makes your topic much more coherent (see the paper https://arxiv.org/abs/2004.03974)
#### ***  Combined TM combines the BoW with SBERT, a process that seems to increase the coherence of the predicted topics (https://arxiv.org/pdf/2004.03974.pdf). 

In [10]:
TOKENIZERS_PARALLELISM= False
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, num_epochs=100, n_components=50)
ctm.fit(training_dataset)  
print(ctm.get_topics(2))

Epoch: [100/100]	 Seen Samples: [1497600/1500000]	Train Loss: 134.98965310642862	Time: 0:00:02.411092: : 100it [04:00,  2.40s/it]
100%|██████████| 235/235 [00:02<00:00, 91.92it/s]


defaultdict(<class 'list'>, {0: ['published', 'book'], 1: ['album', 'released'], 2: ['town', 'england'], 3: ['school', 'high'], 4: ['family', 'found'], 5: ['established', 'education'], 6: ['best', 'known'], 7: ['york', 'new'], 8: ['chinese', 'china'], 9: ['game', 'video'], 10: ['chemical', 'away'], 11: ['film', 'directed'], 12: ['company', 'founded'], 13: ['music', 'band'], 14: ['war', 'class'], 15: ['approximately', 'within'], 16: ['summer', 'world'], 17: ['station', 'line'], 18: ['york', 'new'], 19: ['league', 'season'], 20: ['region', 'municipality'], 21: ['often', 'used'], 22: ['greek', 'bc'], 23: ['approximately', 'within'], 24: ['company', 'owned'], 25: ['street', 'city'], 26: ['states', 'united'], 27: ['french', 'von'], 28: ['played', 'former'], 29: ['american', 'served'], 30: ['river', 'park'], 31: ['university', 'professor'], 32: ['race', 'car'], 33: ['division', 'home'], 34: ['church', 'roman'], 35: ['party', 'political'], 36: ['election', 'member'], 37: ['politician', 'membe

## Save the model for the future references

In [11]:
ctm.save(models_dir="./")


/opt/conda/lib/python3.10/site-packages/contextualized_topic_models/models/ctm.py:640: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn(


## Loading the Model from saved directory

In [12]:
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, num_epochs=100, n_components=50)
file_dir = "/kaggle/working/contextualized_topic_model_nc_50_tpm_0.0_tpv_0.98_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99"
ctm.load("/kaggle/working/contextualized_topic_model_nc_50_tpm_0.0_tpv_0.98_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99/",
                                                                                                      epoch=99)

/opt/conda/lib/python3.10/site-packages/contextualized_topic_models/models/ctm.py:669: Warning: This is an experimental feature that we has not been fully tested. Refer to the following issue:https://github.com/MilaNLProc/contextualized-topic-models/issues/38
  warnings.warn(


After training, now it is the time to look at our topics: we can use the function  get_topic_lists  to get the topics. It also accept a parameter that allows you to select how many words you want to see for each topic. If you look at the topics, you will see that they all make sense and are representative of a collection of documents that comes from Wikipedia (general knowledge).

In [13]:
ctm.get_topic_lists(5)

[['published', 'book', 'magazine', 'books', 'novel'],
 ['album', 'released', 'studio', 'band', 'second'],
 ['town', 'england', 'civil', 'parish', 'city'],
 ['school', 'high', 'public', 'located', 'schools'],
 ['family', 'found', 'mm', 'brown', 'costa'],
 ['established', 'education', 'research', 'institute', 'journal'],
 ['best', 'known', 'director', 'award', 'actor'],
 ['york', 'new', 'american', 'radio', 'music'],
 ['chinese', 'china', 'simplified', 'pinyin', 'traditional'],
 ['game', 'video', 'developed', 'japan', 'series'],
 ['chemical', 'away', 'thus', 'specifically', 'number'],
 ['film', 'directed', 'produced', 'lead', 'stars'],
 ['company', 'founded', 'non', 'profit', 'based'],
 ['music', 'band', 'rock', 'album', 'group'],
 ['war', 'class', 'ii', 'royal', 'world'],
 ['approximately', 'within', 'mi', 'lies', 'west'],
 ['summer', 'world', 'kg', 'winter', 'olympic'],
 ['station', 'line', 'railway', 'train', 'rail'],
 ['york', 'new', 'major', 'baseball', 'american'],
 ['league', 'sea

##  Now we are going to use the testset: we want to predict the topic for unseen documents.

In [14]:
# cear test set
testing_dataset = tp.fit(testing_contextual_documents, testing_bow_documents) # create dataset for the testset
predictions = ctm.get_doc_topic_distribution(testing_dataset, n_samples=10)

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use th

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 79/79 [00:00<00:00, 89.60it/s] 


In [15]:
print(testing_contextual_documents[15])

topic_index = np.argmax(predictions[15])
ctm.get_topic_lists(5)[topic_index]

Dhale (Arabic: الضالع‎‎ Aḍ Ḍāliʿ) province is one of the governorates of Yemen that have been created after the announcement of Yemeni unification. The population of the province accounted for (2.4%) of the total population of the Republic, and allocated administratively into (9) districts. Dali city is the centre of


['iran', 'district', 'families', 'province', 'persian']

In [17]:
testing_dataset = tp.transform(text_for_contextual=testing_contextual_documents, text_for_bow=testing_bow_documents)

# n_sample how many times to sample the distribution (see the doc)
ctm.get_doc_topic_distribution(testing_dataset, n_samples=20) # returns a (n_documents, n_topics) matrix with the topic distribution of each document


/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:438: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use th

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 79/79 [00:00<00:00, 93.60it/s] 


array([[0.02361889, 0.00795612, 0.0063576 , ..., 0.00747827, 0.01270531,
        0.01841428],
       [0.00824461, 0.05588315, 0.00852354, ..., 0.00306182, 0.00807662,
        0.00409952],
       [0.01138107, 0.00965712, 0.00879506, ..., 0.00479927, 0.03324365,
        0.03358855],
       ...,
       [0.02086858, 0.00998342, 0.03541994, ..., 0.02692374, 0.00990599,
        0.03319518],
       [0.02519259, 0.0218731 , 0.0157916 , ..., 0.01391032, 0.01891009,
        0.02534913],
       [0.0325633 , 0.00671137, 0.01068075, ..., 0.00533131, 0.01050693,
        0.00806523]], dtype=float32)

### Topic Predictions
Now we can take a document and see which topic has been assigned to it. Results will obviously change with respect to the documents you are using. For example, let's predict the topic of the first preprocessed document that is talking about a peninsula.

In [18]:
topics_predictions = ctm.get_thetas(testing_dataset, n_samples=5) # get all the topic predictions

100%|██████████| 79/79 [00:00<00:00, 92.84it/s] 


In [19]:
training_bow_documents[0] # see the text of our preprocessed document

'mid peninsula highway proposed across peninsula canadian province ontario although highway connecting hamilton fort south international study published ministry'

In [20]:
topic_number = np.argmax(topics_predictions[0]) # get the topic id of the first document
print(ctm.get_topic_lists(5)[15])
print(ctm.get_topic_lists(5)[topic_number]) #and the topic should be about natural location/places/related things


['approximately', 'within', 'mi', 'lies', 'west']
['university', 'professor', 'arts', 'studies', 'received']
